In [1]:
# imports
from sec_api import QueryApi
import pandas as pd
import json
import pprint
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# functions

# function for querying the api
def query_api(key=str, form_type=str, filed_at=str, ticker=str):
    queryApi = QueryApi(api_key=key)
    query = {
            "query": { "query_string": {
                "query": "formType:" + form_type + " AND filedAt:" + filed_at + " AND ticker:" + ticker
                } },
            "from": "0",
            "size": "100",
            "sort": [{ "filedAt": { "order": "desc" } }]
            }
    filings = queryApi.get_filings(query)

    output = filings['filings']

    return output

#matching function
def fuzzy_match(keyword, string):
    return fuzz.partial_ratio(keyword.lower(), string.lower()) >= 54

In [3]:
output = query_api("a623cd29617f549e2278cb906046e4136f6edd7e861cd072188a05569260f6bb", "11-K", "{2020-01-31 TO *}", '[\"\" TO *]')
# function tidied version
# retrieving the matches from the F-11k filings 

output_list = []

for x in output:
    filing_to_search = x['entities']
    for dict in filing_to_search:
        sic = str(dict['sic'])
        ticker = x['ticker']
        company_name = x['companyName']
        cik = dict['cik']

        key_words = ["healthcare", "financial services", "target return funds", "balanced funds", "mid cap funds"]

        matched_words = [word for word in key_words if fuzzy_match(word, sic)]

        output_dict={}
        if len(matched_words)>0:
            output_dict['ticker']=str(ticker)
            output_dict['company_name']=company_name
            output_dict['cik']=cik
            output_dict['matches']=matched_words
            output_list.append(output_dict)

# print(output_list)

# searching for the tickers with matches the Form-4

form_4_dict = {}


for match in output_list:
    ticker = f'{match["ticker"]}'
    if ticker not in form_4_dict.keys():
        # print('true')
        form_4_output = query_api("a623cd29617f549e2278cb906046e4136f6edd7e861cd072188a05569260f6bb", "4", "{2021-01-31 TO *}", ticker)
        
        for b in form_4_output:
            comp_name = (b['companyName'])
            form_4_comp_name_list = []
            if comp_name not in form_4_comp_name_list:
                form_4_comp_name_list.append(comp_name)
                form_4_dict[ticker]=form_4_comp_name_list

for matched_dict in output_list:
    if matched_dict['ticker'] in form_4_dict.keys():
        ticker = matched_dict['ticker']
        form_4_comp_name_list = form_4_dict[ticker]
        matched_dict['form_4'] = form_4_comp_name_list

df = pd.DataFrame(output_list, index=None)
df['matches'] = df['matches'].astype(str)
display(df)       

['HCM Acquisition Corp']
['HCM Acquisition Corp']
['HCM Acquisition Corp']
['HCM Acquisition Corp']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
['DiamondRock Hospitality Co']
[

,ticker,company_name,cik,matches,form_4
0,HCMA,HCM Acquisition Corp,1845368,['balanced funds'],[HCM Acquisition Corp]
1,DRH,DiamondRock Hospitality Co,1298946,['healthcare'],[DiamondRock Hospitality Co]
2,EBET,"EBET, Inc.",1829966,['financial services'],"[Esports Technologies, Inc.]"
3,TWKS,"Thoughtworks Holding, Inc.",1866550,['financial services'],"[Thoughtworks Holding, Inc.]"
4,GNS,Genius Group Ltd,1847806,['financial services'],NaN
5,VII,7GC & Co. Holdings Inc.,1826011,['balanced funds'],NaN
6,TFII,TFI International Inc.,1588823,['financial services'],NaN
7,ARTE,Artemis Strategic Investment Corp,1839990,['balanced funds'],[Artemis Strategic Investment Corp]
8,PFTA,PORTAGE FINTECH ACQUISITION CORP.,1853580,['balanced funds'],[PORTAGE FINTECH ACQUISITION CORP.]
9,FRLA,Fortune Rise Acquisition Corp,1849294,['balanced funds'],[Fortune Rise Acquisition Corp]
